In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import difflib
from collections import defaultdict

In [ ]:
# class EcoCartRecommender:
#     def __init__(self, product_file, event_file, profile_file):
#         self.products = pd.read_csv(product_file)
#         self.events = pd.read_csv(event_file)
#         self.profiles = pd.read_csv(profile_file)

#         self.popularity = self._compute_popularity()
#         self.model = self._train_score_model()

#         self.tier_weights = {
#             'Bronze': {'eco_score': 1.0, 'refurbished': 1.0, 'popularity': 1.0},
#             'Silver': {'eco_score': 1.2, 'refurbished': 1.2, 'popularity': 1.0},
#             'Gold': {'eco_score': 1.4, 'refurbished': 1.4, 'popularity': 0.9},
#             'Platinum': {'eco_score': 1.6, 'refurbished': 1.6, 'popularity': 0.8}
#         }

#     def _compute_popularity(self):
#         pop = self.events.groupby('itemID')['event_type'].value_counts().unstack().fillna(0)
#         pop['popularity_score'] = 1 * pop.get('view', 0) + 2 * pop.get('cart', 0) + 3 * pop.get('purchase', 0)
#         return pop['popularity_score']

#     def _generate_training_data(self):
#         merged = self.events.merge(self.products, on='itemID')
#         merged = merged.merge(self.profiles, on='user_id')

#         merged['popularity'] = merged['itemID'].map(self.popularity).fillna(0)
#         merged['eco_score'] = merged['Total(kg CO2e)']
#         merged['eco_score_norm'] = (merged['eco_score'] - merged['eco_score'].min()) / (merged['eco_score'].max() - merged['eco_score'].min())

#         merged['category_match'] = 1
#         merged['refurbished'] = merged['refurbished_available']
#         merged['price_penalty'] = merged['price'] / merged['avg_spend']

#         merged['label'] = merged['event_type'].apply(lambda x: 1 if x in ['purchase', 'cart'] else 0)

#         features = merged[['popularity', 'eco_score_norm', 'category_match', 'refurbished', 'price_penalty']]
#         labels = merged['label']
#         return features, labels

#     def _train_score_model(self):
#         X, y = self._generate_training_data()
#         model = LogisticRegression()
#         model.fit(X, y)
#         return model

#     def _compute_features(self, product_row, user_profile, user_tier, green_mode=True):
#         pop = self.popularity.get(product_row['itemID'], 0)
#         eco_score = product_row['Total(kg CO2e)']
#         eco_score_norm = (eco_score - self.products['Total(kg CO2e)'].min()) / (self.products['Total(kg CO2e)'].max() - self.products['Total(kg CO2e)'].min())
#         category_match = 1
#         refurbished = product_row['refurbished_available']
#         price_penalty = product_row['price'] / user_profile['avg_spend']

#         weights = self.tier_weights.get(user_tier, self.tier_weights['Bronze']) if green_mode else {'eco_score': 1.0, 'refurbished': 1.0, 'popularity': 1.0}


#         return pd.DataFrame([[pop * weights['popularity'], eco_score_norm * weights['eco_score'], category_match, refurbished * weights['refurbished'], price_penalty]], 
#                             columns=['popularity', 'eco_score_norm', 'category_match', 'refurbished', 'price_penalty'])

#     def recommend_for_user(self, user_id, top_n=10, green_mode=True, search_query=None):
#         user_profile = self.profiles[self.profiles['user_id'] == user_id].squeeze()
#         reward_engine = RewardEngine("Dataset/Synthetic_Dataset/user_green_engagement_logs.csv") # Equivalent to the user behavior file
#         user_tier = reward_engine.get_user_tier(user_id)

#         scores = []

#         if search_query:
#             search_query_lower = search_query.lower()
#             product_types = self.products['Product'].dropna().unique()
#             match = difflib.get_close_matches(search_query_lower, product_types, n=1, cutoff=0.3)
#             matched_category = match[0] if match else None

#             main_products = self.products[self.products['Product'].str.lower() == matched_category.lower()] if matched_category else pd.DataFrame()
#             alt_products = self.products[self.products['Product'].str.lower() != matched_category.lower()] if matched_category else self.products

#             filtered_products = pd.concat([main_products, alt_products])
#         else:
#             filtered_products = self.products

#         for _, row in filtered_products.iterrows():
#             features = self._compute_features(row, user_profile, user_tier, green_mode)
#             score = self.model.predict_proba(features)[0][1]
#             scores.append((score, row))

#         if search_query and matched_category:
#             same_cat = [(s, r) for s, r in scores if r['Product'].lower() == matched_category.lower()]
#             same_cat_sorted = sorted(same_cat, key=lambda x: (-x[0], x[1]['Total(kg CO2e)']))

#             same_cat_ids = {r['itemID'] for _, r in same_cat}
#             same_cat_low = [(s, r) for s, r in scores if r['Product'].lower() == matched_category.lower() and r['itemID'] not in same_cat_ids]
#             same_cat_low_sorted = sorted(same_cat_low, key=lambda x: x[1]['Total(kg CO2e)'])

#             other_cat = [(s, r) for s, r in scores if r['Product'].lower() != matched_category.lower()]
#             other_cat_sorted = sorted(other_cat, key=lambda x: x[1]['Total(kg CO2e)'])

#             top_items = [r for _, r in same_cat_sorted[:top_n]]
#             remaining = top_n - len(top_items)
#             if remaining > 0:
#                 top_items += [r for _, r in same_cat_low_sorted[:remaining]]
#                 remaining = top_n - len(top_items)
#                 if remaining > 0:
#                     top_items += [r for _, r in other_cat_sorted[:remaining]]
#         else:
#             if green_mode:
#                 sorted_items = sorted(scores, key=lambda x: (-x[0], x[1]['Total(kg CO2e)']))
#             else:
#                 sorted_items = sorted(scores, key=lambda x: -x[0])
#             top_items = [r for _, r in sorted_items[:top_n]]

#         top_items_df = pd.DataFrame(top_items)
#         return top_items_df[['itemID', 'Product', 'Variety/Size', 'Brand', 'price', 'Total(kg CO2e)', 'refurbished_available']]


# class RewardEngine:
#     def __init__(self, behavior_file):
#         self.behavior = pd.read_csv(behavior_file)

#     def calculate_rewards(self):
#         user_rewards = defaultdict(int)

#         for _, row in self.behavior.iterrows():
#             eco_score = row['eco_score']
#             base_reward = max(0, 450 - eco_score) / 45

#             if row['refurbished']:
#                 base_reward += 2
#             if row['delivery_type'] == 'delayed':
#                 base_reward += 1
#             if row['used_green_toggle']:
#                 base_reward += 0.5

#             user_rewards[row['user_id']] += round(base_reward, 2)

#         return pd.DataFrame(list(user_rewards.items()), columns=['user_id', 'eco_points'])

#     def get_user_tier(self, user_id):
#         df = self.calculate_rewards()
#         points = df[df['user_id'] == user_id]['eco_points']
#         if points.empty:
#             return 'Bronze'
#         score = points.values[0]

#         if score >= 100:
#             return 'Platinum'
#         elif score >= 50:
#             return 'Gold'
#         elif score >= 20:
#             return 'Silver'
#         else:
#             return 'Bronze'
        
#     def get_user_perks(self, user_id):
#         tier = self.get_user_tier(user_id)

#         perks = {
#             'Bronze': [
#                 "Start earning by using green mode toggle.",
#                 "Buy refurbished or delay delivery to earn more points."
#             ],
#             'Silver': [
#                 "5% discount on refurbished items.",
#                 "Bonus 0.5 pts for every green mode interaction."
#             ],
#             'Gold': [
#                 "10% discount on green deliveries.",
#                 "Early access to eco-friendly offers."
#             ],
#             'Platinum': [
#                 "Free shipping on refurbished products.",
#                 "Exclusive platinum-only deals every month.",
#                 "15% discount on green delivery items."
#             ]
#         }

#         return perks.get(tier, [])


In [2]:
## Better version with vectorized operations and improved search handling

class EcoCartRecommender:
    def __init__(self, product_file, event_file, profile_file):
        self.products = pd.read_csv(product_file)
        self.events = pd.read_csv(event_file)
        self.profiles = pd.read_csv(profile_file)

        self.popularity = self._compute_popularity()
        self.model = self._train_score_model()

        self.tier_weights = {
            'Bronze': {'eco_score': 1.0, 'refurbished': 1.0, 'popularity': 1.0},
            'Silver': {'eco_score': 1.2, 'refurbished': 1.2, 'popularity': 1.0},
            'Gold': {'eco_score': 1.4, 'refurbished': 1.4, 'popularity': 0.9},
            'Platinum': {'eco_score': 1.6, 'refurbished': 1.6, 'popularity': 0.8}
        }

    def _compute_popularity(self):
        pop = self.events.groupby('itemID')['event_type'].value_counts().unstack().fillna(0)
        pop['popularity_score'] = 1 * pop.get('view', 0) + 2 * pop.get('cart', 0) + 3 * pop.get('purchase', 0)
        return pop['popularity_score']

    def _generate_training_data(self):
        merged = self.events.merge(self.products, on='itemID')
        merged = merged.merge(self.profiles, on='user_id')

        merged['popularity'] = merged['itemID'].map(self.popularity).fillna(0)
        merged['eco_score'] = merged['Total(kg CO2e)']
        merged['eco_score_norm'] = (merged['eco_score'] - merged['eco_score'].min()) / (merged['eco_score'].max() - merged['eco_score'].min())

        merged['category_match'] = 1
        merged['refurbished'] = merged['refurbished_available']
        merged['price_penalty'] = merged['price'] / merged['avg_spend']

        merged['label'] = merged['event_type'].apply(lambda x: 1 if x in ['purchase', 'cart'] else 0)

        features = merged[['popularity', 'eco_score_norm', 'category_match', 'refurbished', 'price_penalty']]
        labels = merged['label']
        return features, labels

    def _train_score_model(self):
        X, y = self._generate_training_data()
        model = LogisticRegression()
        model.fit(X, y)
        return model

    def _compute_features(self, product_row, user_profile, user_tier, green_mode=True):
        pop = self.popularity.get(product_row['itemID'], 0)
        eco_score = product_row['Total(kg CO2e)']
        eco_score_norm = (eco_score - self.products['Total(kg CO2e)'].min()) / (self.products['Total(kg CO2e)'].max() - self.products['Total(kg CO2e)'].min())
        category_match = 1
        refurbished = product_row['refurbished_available']
        price_penalty = product_row['price'] / user_profile['avg_spend']

        weights = self.tier_weights.get(user_tier, self.tier_weights['Bronze']) if green_mode else {'eco_score': 1.0, 'refurbished': 1.0, 'popularity': 1.0}


        return pd.DataFrame([[pop * weights['popularity'], eco_score_norm * weights['eco_score'], category_match, refurbished * weights['refurbished'], price_penalty]], 
                            columns=['popularity', 'eco_score_norm', 'category_match', 'refurbished', 'price_penalty'])

    def recommend_for_user(self, user_id, top_n=10, green_mode=True, search_query=None):
        user_profile = self.profiles[self.profiles['user_id'] == user_id].squeeze()
        reward_engine = RewardEngine("Dataset/Synthetic_Dataset/user_green_engagement_logs.csv")
        user_tier = reward_engine.get_user_tier(user_id)

        # Filter products by search query if provided
        if search_query:
            search_query_lower = search_query.lower()
            product_types = self.products['Product'].dropna().unique()
            match = difflib.get_close_matches(search_query_lower, product_types, n=1, cutoff=0.3)
            matched_category = match[0] if match else None

            main_products = self.products[self.products['Product'].str.lower() == matched_category.lower()] if matched_category else pd.DataFrame()
            alt_products = self.products[self.products['Product'].str.lower() != matched_category.lower()] if matched_category else self.products

            filtered_products = pd.concat([main_products, alt_products])
        else:
            filtered_products = self.products

        # Vectorized feature computation
        pop = filtered_products['itemID'].map(self.popularity).fillna(0)
        eco_score = filtered_products['Total(kg CO2e)']
        eco_score_norm = (eco_score - self.products['Total(kg CO2e)'].min()) / (self.products['Total(kg CO2e)'].max() - self.products['Total(kg CO2e)'].min())
        category_match = 1
        refurbished = filtered_products['refurbished_available']
        price_penalty = filtered_products['price'] / user_profile['avg_spend']

        weights = self.tier_weights.get(user_tier, self.tier_weights['Bronze']) if green_mode else {'eco_score': 1.0, 'refurbished': 1.0, 'popularity': 1.0}

        features = pd.DataFrame({
            'popularity': pop * weights['popularity'],
            'eco_score_norm': eco_score_norm * weights['eco_score'],
            'category_match': category_match,
            'refurbished': refurbished * weights['refurbished'],
            'price_penalty': price_penalty
        })

        # Batch prediction
        scores = self.model.predict_proba(features)[:, 1]
        filtered_products = filtered_products.copy()
        filtered_products['score'] = scores

        if search_query and matched_category:
            same_cat = filtered_products[filtered_products['Product'].str.lower() == matched_category.lower()]
            same_cat_sorted = same_cat.sort_values(['score', 'Total(kg CO2e)'], ascending=[False, True])

            same_cat_ids = set(same_cat_sorted['itemID'])
            same_cat_low = filtered_products[(filtered_products['Product'].str.lower() == matched_category.lower()) & (~filtered_products['itemID'].isin(same_cat_ids))]
            same_cat_low_sorted = same_cat_low.sort_values('Total(kg CO2e)')

            other_cat = filtered_products[filtered_products['Product'].str.lower() != matched_category.lower()]
            other_cat_sorted = other_cat.sort_values('Total(kg CO2e)')

            top_items = pd.concat([
                same_cat_sorted.head(top_n),
                same_cat_low_sorted.head(max(0, top_n - len(same_cat_sorted))),
                other_cat_sorted.head(max(0, top_n - len(same_cat_sorted) - len(same_cat_low_sorted)))
            ]).head(top_n)
        else:
            if green_mode:
                top_items = filtered_products.sort_values(['score', 'Total(kg CO2e)'], ascending=[False, True]).head(top_n)
            else:
                top_items = filtered_products.sort_values('score', ascending=False).head(top_n)

        return top_items[['itemID', 'Product', 'Variety/Size', 'Brand', 'price', 'Total(kg CO2e)', 'refurbished_available']]


class RewardEngine:
    def __init__(self, behavior_file):
        self.behavior = pd.read_csv(behavior_file)

    def calculate_rewards(self):
        user_rewards = defaultdict(int)

        for _, row in self.behavior.iterrows():
            eco_score = row['eco_score']
            base_reward = max(0, 450 - eco_score) / 45

            if row['refurbished']:
                base_reward += 2
            if row['delivery_type'] == 'delayed':
                base_reward += 1
            if row['used_green_toggle']:
                base_reward += 0.5

            user_rewards[row['user_id']] += round(base_reward, 2)

        return pd.DataFrame(list(user_rewards.items()), columns=['user_id', 'eco_points'])

    def get_user_tier(self, user_id):
        df = self.calculate_rewards()
        points = df[df['user_id'] == user_id]['eco_points']
        if points.empty:
            return 'Bronze'
        score = points.values[0]

        if score >= 180:
            return 'Platinum'
        elif score >= 140:
            return 'Gold'
        elif score >= 100:
            return 'Silver'
        else:
            return 'Bronze'
        ## These Values are set based on after visualization and analysis of the data
        
    def get_user_perks(self, user_id):
        tier = self.get_user_tier(user_id)

        perks = {
            'Bronze': [
                "Start earning by using green mode toggle.",
                "Buy refurbished or delay delivery to earn more points."
            ],
            'Silver': [
                "5% discount on refurbished items.",
                "Bonus 0.5 pts for every green mode interaction."
            ],
            'Gold': [
                "10% discount on green deliveries.",
                "Early access to eco-friendly offers."
            ],
            'Platinum': [
                "Free shipping on refurbished products.",
                "Exclusive platinum-only deals every month.",
                "15% discount on green delivery items."
            ]
        }

        return perks.get(tier, [])


In [3]:
recommender = EcoCartRecommender(
    product_file="Dataset/Synthetic_Dataset/products_catalog.csv",
    event_file="Dataset/Synthetic_Dataset/user_events.csv",
    profile_file="Dataset/Synthetic_Dataset/user_profiles.csv"
)

reward_engine = RewardEngine("Dataset/Synthetic_Dataset/user_green_engagement_logs.csv") ## Equivalent to the user behavior file

In [4]:
user_id = 101  # Change as needed

In [5]:
print("\n--- Green Mode Recommendations ---")
print(recommender.recommend_for_user(user_id, green_mode=True))


--- Green Mode Recommendations ---
     itemID                Product Variety/Size      Brand  price  \
393     393   Air Conditioner (AC)        2 Ton    Hitachi  48040   
135     135  Refrigerator (Fridge)        500 L     Godrej  55454   
245     245        Television (TV)      43 inch     Xiaomi  32063   
132     132        Washing Machine         8 kg    Samsung  29627   
362     362   Air Conditioner (AC)        2 Ton     Voltas  40470   
60       60  Refrigerator (Fridge)        340 L         LG  36613   
468     468  Refrigerator (Fridge)        300 L     Godrej  28532   
471     471   Air Conditioner (AC)      1.5 Ton  Blue Star  49620   
79       79  Refrigerator (Fridge)        500 L    Samsung  43541   
425     425  Refrigerator (Fridge)        240 L  Panasonic  23022   

     Total(kg CO2e)  refurbished_available  
393          416.30                      1  
135          311.85                      1  
245          161.71                      1  
132          171.00     

In [6]:
print("\n--- Normal Mode Recommendations ---")
print(recommender.recommend_for_user(user_id, green_mode=False))


--- Normal Mode Recommendations ---
     itemID                Product Variety/Size      Brand  price  \
245     245        Television (TV)      43 inch     Xiaomi  32063   
132     132        Washing Machine         8 kg    Samsung  29627   
135     135  Refrigerator (Fridge)        500 L     Godrej  55454   
393     393   Air Conditioner (AC)        2 Ton    Hitachi  48040   
241     241    Geyser/Water Heater          6 L     Racold   3853   
20       20                 Cooler         20 L   Symphony   5549   
60       60  Refrigerator (Fridge)        340 L         LG  36613   
383     383   Air Conditioner (AC)        1 Ton         LG  34524   
488     488                   Fans      48 inch    Havells   2860   
479     479        Washing Machine         6 kg  Whirlpool  20203   

     Total(kg CO2e)  refurbished_available  
245          161.71                      1  
132          171.00                      1  
135          311.85                      1  
393          416.30    

In [7]:
print("\n--- Green Mode Recommendations ---")
print(recommender.recommend_for_user(user_id, green_mode=True, search_query='Washing Machine'))


--- Green Mode Recommendations ---
     itemID          Product Variety/Size      Brand  price  Total(kg CO2e)  \
132     132  Washing Machine         8 kg    Samsung  29627          171.00   
479     479  Washing Machine         6 kg  Whirlpool  20203          115.20   
34       34  Washing Machine         9 kg         LG  30855          211.68   
452     452  Washing Machine         8 kg    Samsung  34021          171.00   
397     397  Washing Machine         8 kg      Bosch  33907          172.80   
232     232  Washing Machine         7 kg        IFB  29185          139.68   
438     438  Washing Machine         7 kg  Whirlpool  20944          138.24   
147     147  Washing Machine         7 kg      Bosch  29009          138.24   
93       93  Washing Machine         6 kg         LG  23777          117.60   
59       59  Washing Machine         8 kg     Godrej  36553          178.20   

     refurbished_available  
132                      1  
479                      1  
34     

In [16]:
print("\n--- Normal Mode Recommendations ---")
print(recommender.recommend_for_user(user_id, green_mode=False, search_query='Washing Machine'))


--- Normal Mode Recommendations ---
     itemID          Product Variety/Size      Brand  price  Total(kg CO2e)  \
132     132  Washing Machine         8 kg    Samsung  29627          171.00   
479     479  Washing Machine         6 kg  Whirlpool  20203          115.20   
452     452  Washing Machine         8 kg    Samsung  34021          171.00   
34       34  Washing Machine         9 kg         LG  30855          211.68   
397     397  Washing Machine         8 kg      Bosch  33907          172.80   
93       93  Washing Machine         6 kg         LG  23777          117.60   
232     232  Washing Machine         7 kg        IFB  29185          139.68   
59       59  Washing Machine         8 kg     Godrej  36553          178.20   
438     438  Washing Machine         7 kg  Whirlpool  20944          138.24   
147     147  Washing Machine         7 kg      Bosch  29009          138.24   

     refurbished_available  
132                      1  
479                      1  
452   

In [8]:
print("\n--- Green Mode Recommendations ---")
print(recommender.recommend_for_user(user_id, green_mode=True, search_query='Fans'))


--- Green Mode Recommendations ---
     itemID Product Variety/Size     Brand  price  Total(kg CO2e)  \
488     488    Fans      48 inch   Havells   2860           24.00   
478     478    Fans      48 inch     Bajaj   2663           24.24   
5         5    Fans      48 inch    Orient   2951           26.16   
386     386    Fans      48 inch  Atomberg   2294           25.20   
463     463    Fans      48 inch     Bajaj   2767           24.24   
269     269    Fans      52 inch  Crompton   3251           30.50   
237     237    Fans      48 inch     Bajaj   2335           24.24   
69       69    Fans      52 inch  Atomberg   3081           32.02   
177     177    Fans      48 inch    Orient   2023           26.16   
292     292    Fans      48 inch   Havells   2033           24.00   

     refurbished_available  
488                      1  
478                      1  
5                        1  
386                      1  
463                      1  
269                      0  
2

In [9]:
print("\n--- Green Mode Recommendations ---")
print(recommender.recommend_for_user(user_id, green_mode=True, search_query='Water Purifier'))


--- Green Mode Recommendations ---
     itemID         Product Variety/Size      Brand  price  Total(kg CO2e)  \
108     108  Water Purifier     RO+UV+UF     Pureit  19224           90.16   
51       51  Water Purifier     RO+UV+UF        HUL  13222           92.00   
369     369  Water Purifier        RO+UV        HUL   8296           70.00   
462     462  Water Purifier     RO+UV+UF  Aquaguard  13526           93.84   
262     262  Water Purifier     RO+UV+UF  Blue Star  10446           92.92   
181     181  Water Purifier     RO+UV+UF        HUL  14616           92.00   
234     234  Water Purifier     RO+UV+UF       Kent  18954           92.00   
319     319  Water Purifier     RO+UV+UF    Livpure  11139           96.60   
179     179  Water Purifier     RO+UV+UF    Livpure  13241           96.60   
109     109  Water Purifier     RO+UV+UF  Blue Star  12226           92.92   

     refurbished_available  
108                      1  
51                       0  
369               

In [10]:
print("\n--- Green Mode Recommendations ---")
print(recommender.recommend_for_user(user_id, green_mode=True, search_query='Television (TV)'))


--- Green Mode Recommendations ---
     itemID          Product Variety/Size    Brand  price  Total(kg CO2e)  \
245     245  Television (TV)      43 inch   Xiaomi  32063          161.71   
1         1  Television (TV)      65 inch     Sony  65187          338.40   
106     106  Television (TV)      43 inch       Vu  38538          172.70   
226     226  Television (TV)      50 inch   Xiaomi  36715          200.85   
156     156  Television (TV)      43 inch     Sony  29865          188.40   
101     101  Television (TV)      65 inch  Samsung  64474          267.90   
445     445  Television (TV)      50 inch       Vu  46306          214.50   
219     219  Television (TV)      65 inch       Vu  71175          310.20   
214     214  Television (TV)      50 inch   Xiaomi  49242          200.85   
131     131  Television (TV)      32 inch       LG  20524          127.40   

     refurbished_available  
245                      1  
1                        0  
106                      1  


In [11]:
print("\n--- Green Mode Recommendations ---")
print(recommender.recommend_for_user(user_id, green_mode=True, search_query='Refrigerator (Fridge)'))


--- Green Mode Recommendations ---
     itemID                Product Variety/Size      Brand  price  \
135     135  Refrigerator (Fridge)        500 L     Godrej  55454   
60       60  Refrigerator (Fridge)        340 L         LG  36613   
468     468  Refrigerator (Fridge)        300 L     Godrej  28532   
79       79  Refrigerator (Fridge)        500 L    Samsung  43541   
425     425  Refrigerator (Fridge)        240 L  Panasonic  23022   
157     157  Refrigerator (Fridge)        240 L     Godrej  27424   
144     144  Refrigerator (Fridge)        240 L  Whirlpool  25267   
229     229  Refrigerator (Fridge)        190 L      Haier  17227   
322     322  Refrigerator (Fridge)        500 L  Whirlpool  41721   
238     238  Refrigerator (Fridge)        340 L  Whirlpool  31609   

     Total(kg CO2e)  refurbished_available  
135          311.85                      1  
60           266.56                      1  
468          242.55                      1  
79           299.25     

In [12]:
print(f"User {user_id} Tier:", reward_engine.get_user_tier(user_id))

User 101 Tier: Gold


In [13]:
print("--- Perks for You ---")
for perk in reward_engine.get_user_perks(user_id):
    print("•", perk)

--- Perks for You ---
• 10% discount on green deliveries.
• Early access to eco-friendly offers.


In [14]:
print("\n--- User Eco Points ---")
print(reward_engine.calculate_rewards())


--- User Eco Points ---
     user_id  eco_points
0        475      170.02
1        479      102.27
2        323      154.14
3        345      118.39
4        353      116.89
..       ...         ...
495       99      114.18
496      220       92.79
497       97      124.66
498      277      150.80
499       81       84.66

[500 rows x 2 columns]


In [15]:
dff = reward_engine.calculate_rewards()

dff

,user_id,eco_points
0,475,170.02
1,479,102.27
2,323,154.14
3,345,118.39
4,353,116.89
...,...,...
495,99,114.18
496,220,92.79
497,97,124.66
498,277,150.80
